In [1]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
import pandas as pd
import numpy as np
import utils.public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import (
    pack_padded_sequence,
    pack_sequence,
    unpack_sequence,
    unpad_sequence,
)
import torch
from tqdm.notebook import trange, tqdm
import torch.nn as nn
import torch.optim as optim
import wandb
import utils.torch_classes
from utils.model_saver import model_saver_wandb as model_saver
import utils.training_testing
import utils.training_testing_dual_optim
from itertools import combinations
import gc
from sklearn.decomposition import PCA
import sys
import lightgbm as lgb
import time
from sklearn.preprocessing import OneHotEncoder
import utils.training_testing_double
from utils.constants import *
import importlib

In [2]:
%env "WANDB_NOTEBOOK_NAME" os.path.basename(__file__)

env: "WANDB_NOTEBOOK_NAME"=os.path.basename(__file__)


In [3]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [4]:
torch.__version__

'2.1.1'

In [5]:
if torch.cuda.is_available():
    device = torch.device(
        "cuda:0"
        
    )  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [6]:
train = pd.read_csv("data/train.csv")
weights_df = pd.DataFrame(
    data=list(zip(range(0, 201), WEIGHTS)), columns=["stock_id", "index_weight"]
)
train = train.merge(weights_df, on="stock_id")

In [7]:
train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'index_weight'],
      dtype='object')

In [8]:
def generate_prev_race(df_in, df_g, rolling_window=10, factor=""):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f"wap_t-60"] = df_g["wap"].shift(6)
    df[f"target_t-60"] = df_g["target"].shift(6)
    df[f"initial_wap"] = df_g["wap_calc"].transform("first")
    df[f"initial_bid_size"] = df_g["bid_size"].transform("first")
    df[f"initial_ask_size"] = df_g["ask_size"].transform("first")
    cols = ["bid_price", "ask_price", "bid_size", "ask_size", "wap"]
    for i in cols:
        df[f"{i}_t-60"] = df_g[i].shift(-6)
    for i in cols:
        df[f"{i}_t10"] = df_g[i].shift(1)

    return df

In [9]:
def generate_index(df_in, df_g, rolling_window=10, factor=""):
    df = df_in.copy()
    df[f"index_wap"] = df_g["wap_weighted"].transform("mean")
    return df


def generate_index_2(df_in, df_g, rolling_window=10, factor=""):
    df = df_in.copy()
    df[f"index_wap_t-60"] = df_g["index_wap"].shift(6)
    df[f"index_wap_init"] = df_g["index_wap"].transform("first")
    return df


def generate_index_3(df_in, df_g, rolling_window=10, factor=""):
    df = df_in.copy()
    df[f"index_wap_t-60"] = df_g["index_wap_move_to_init"].shift(6)
    return df

In [10]:
train["wap_calc"] = (
    train["bid_price"] * train["ask_size"] + train["ask_price"] * train["bid_size"]
) / (train["ask_size"] + train["bid_size"])

train["wap_weighted"] = train["wap"] * train["index_weight"]
train_g = train.groupby(["stock_id", "date_id"])
train = generate_prev_race(train, train_g)
train["delta_wap"] = train["wap"] / train["wap_t-60"]

train_g = train.groupby(["seconds_in_bucket", "date_id"])
train = generate_index(train, train_g)


train["wap_move_to_init"] = train["wap_calc"] / train["initial_wap"]
train_g = train.groupby(["date_id"])
train = generate_index_2(train, train_g)

train["index_wap_move_to_init"] = train["index_wap"] / train["index_wap_init"]
train_g = train.groupby(["date_id"])
train = generate_index_3(train, train_g)

In [11]:
train["target_calc"] = (
    -(
        (train["wap_t-60"] / train["wap"])
        - (train["index_wap_t-60"] / train["index_wap_move_to_init"])
    )
    * 10000
)
train["target_delta"] = train["target_t-60"] - train["target_calc"]

In [12]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id"
median_vol = median_vol[["overall_medvol", "first5min_medvol", "last5min_medvol"]]
median_sizes = (
    train.groupby("stock_id")["bid_size"].median()
    + train.groupby("stock_id")["ask_size"].median()
)
std_sizes = (
    train.groupby("stock_id")["bid_size"].median()
    + train.groupby("stock_id")["ask_size"].median()
)

In [13]:
train["bid_price_target"] = train["bid_price"] - train["bid_price_t-60"]
train["bid_price_t-60"] = train["bid_price_target"] * 10_000

train["wap_target"] = train["wap"] - train["wap_t-60"]
train["wap_price_t-60"] = train["wap_target"] * 10_000

targets = ["wap", "bid_price", "ask_price"]
for i in targets:
    train[f"{i}_prev_move"] = (train[f"{i}"] - train[f"{i}_t10"]).fillna(0) * 10000

train["ask_price_target"] = train["ask_price"] - train["ask_price_t-60"]
train["ask_price_t-60"] = train["ask_price_target"] * 10_000

In [14]:
def feat_eng(df):
    cols = [c for c in df.columns if c not in ["row_id", "time_id"]]
    df = df[cols]
    df = df.merge(median_vol, how="left", left_on="stock_id", right_index=True)

    df["bid_plus_ask_sizes"] = df["bid_size"] + train["ask_size"]
    #     df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df["std_size"] = df["stock_id"].map(std_sizes.to_dict())
    #     df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
    df["imbalance_ratio"] = df["imbalance_size"] / df["matched_size"]

    df["imb_s1"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["imb_s2"] = df.eval(
        "(imbalance_size-matched_size)/(matched_size+imbalance_size)"
    )

    df["ask_x_size"] = df.eval("ask_size*ask_price")
    df["bid_x_size"] = df.eval("bid_size*bid_price")

    df["ask_minus_bid"] = df["ask_x_size"] - df["bid_x_size"]
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])


    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    

    prices = [
        "reference_price",
        "far_price",
        "near_price",
        "ask_price",
        "bid_price",
        "wap",
    ]

    for c in combinations(prices, 2):
        df[f"{c[0]}_minus_{c[1]}"] = (df[f"{c[0]}"] - df[f"{c[1]}"]).astype(np.float32)
        df[f"{c[0]}_times_{c[1]}"] = (df[f"{c[0]}"] * df[f"{c[1]}"]).astype(np.float32)
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]}-{c[1]})/({c[0]}+{c[1]})")

    for c in combinations(prices, 3):
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1) - min_ - max_

        df[f"{c[0]}_{c[1]}_{c[2]}_imb2"] = (max_ - mid_) / (mid_ - min_)

    df.drop(
        columns=[
            # 'date_id',
            "reference_price_far_price_imb",
            "reference_price_minus_near_price",
            "reference_price_near_price_imb",
            "far_price_near_price_imb",
            "far_price_ask_price_imb",
            "far_price_bid_price_imb",
            "far_price_minus_wap",
            "std_size",
            "bid_size_over_ask_size",
            "ask_price_bid_price_imb",
            "near_price_times_wap",
        ],
        inplace=True,
    )

    # gc.collect()

    df.replace([np.inf, -np.inf], 0, inplace=True)

    return df

In [15]:
y = train["target"].values
X = feat_eng(train)
prices = [
    c for c in X.columns if ("price" in c) and ("target" not in c) and ("60" not in c)
]
print(prices)

['reference_price', 'far_price', 'near_price', 'bid_price', 'ask_price', 'bid_price_t10', 'ask_price_t10', 'bid_price_prev_move', 'ask_price_prev_move', 'mid_price', 'price_spread', 'price_pressure', 'bid_price_over_ask_price', 'reference_price_minus_far_price', 'reference_price_times_far_price', 'reference_price_times_near_price', 'reference_price_minus_ask_price', 'reference_price_times_ask_price', 'reference_price_ask_price_imb', 'reference_price_minus_bid_price', 'reference_price_times_bid_price', 'reference_price_bid_price_imb', 'reference_price_minus_wap', 'reference_price_times_wap', 'reference_price_wap_imb', 'far_price_minus_near_price', 'far_price_times_near_price', 'far_price_minus_ask_price', 'far_price_times_ask_price', 'far_price_minus_bid_price', 'far_price_times_bid_price', 'far_price_times_wap', 'far_price_wap_imb', 'near_price_minus_ask_price', 'near_price_times_ask_price', 'near_price_ask_price_imb', 'near_price_minus_bid_price', 'near_price_times_bid_price', 'near_p

In [16]:
prices = [
    c for c in X.columns if ("price" in c) and ("target" not in c) and ("60" not in c)
]
# prices = [c for c in train.columns if 'price' in c]
pca_prices = PCA(n_components=1)
X["pca_prices"] = pca_prices.fit_transform(X[prices].fillna(1))

In [17]:
lgbm = lgb.Booster(model_file="data/lgbm_model_new_t60_train_target.lgb")
X_train = X[lgbm_columns]
lgbm_preds = lgbm.predict(X_train)
X["lgbm_preds_target"] = lgbm_preds

In [18]:
lgbm = lgb.Booster(model_file="data/lgbm_model_new_t60_train_wap.lgb")
X_train = X[lgbm_columns]
lgbm_preds = lgbm.predict(X_train)
X["lgbm_preds_wap"] = lgbm_preds

In [19]:
lgbm.feature_name()

['stock_id',
 'seconds_in_bucket',
 'imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'near_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'index_weight',
 'wap_calc',
 'wap_weighted',
 'initial_wap',
 'initial_bid_size',
 'initial_ask_size',
 'bid_price_t10',
 'ask_price_t10',
 'bid_size_t10',
 'ask_size_t10',
 'wap_t10',
 'index_wap',
 'wap_move_to_init',
 'index_wap_init',
 'index_wap_move_to_init',
 'wap_prev_move',
 'bid_price_prev_move',
 'ask_price_prev_move',
 'overall_medvol',
 'first5min_medvol',
 'last5min_medvol',
 'bid_plus_ask_sizes',
 'imbalance_ratio',
 'imb_s1',
 'imb_s2',
 'ask_x_size',
 'bid_x_size',
 'ask_minus_bid',
 'bid_price_over_ask_price',
 'reference_price_minus_far_price',
 'reference_price_times_far_price',
 'reference_price_times_near_price',
 'reference_price_minus_ask_price',
 'reference_price_times_ask_price',
 'reference_price_ask_price_imb',
 'reference_price_minus_bid_price',
 'r

In [20]:
del train

In [21]:
importlib.reload(utils.constants)

<module 'utils.constants' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\utils\\constants.py'>

In [22]:
del stat_cols
from utils.constants import *
importlib.reload(utils.constants)

<module 'utils.constants' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\utils\\constants.py'>

In [23]:
importlib.reload(utils.constants)

<module 'utils.constants' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\utils\\constants.py'>

In [24]:
X["stats"] = np.split(
    np.nan_to_num(X[stat_cols].to_numpy(), nan=-1), indices_or_sections=len(X)
)

In [25]:
sys.getsizeof(X) / (1024**3)

4.975814152508974

In [26]:
X["stats"] = X["stats"].apply(lambda x: x.reshape(-1))
# print(X["stats"].head(10))
# X.to_feather('train_data_with_features.fth')

In [27]:
X["wap_category"] = pd.qcut(X["wap_price_t-60"], q=7)
X["target_category"] = pd.qcut(X["target"], q=7)

In [28]:
X["wap_category2"] = pd.qcut(X["wap_price_t-60"], q=2)
X["target_category2"] = pd.qcut(X["target"], q=2)

In [29]:
X["wap_category3"] = pd.qcut(X["wap_price_t-60"], q=15)
X["target_category3"] = pd.qcut(X["target"], q=15)

In [30]:
weights = (X["wap_category"].value_counts(sort=False).reset_index().sort_values("wap_category"))
weights["norm_count"] = 1 - (weights["count"] / weights["count"].sum())
weight = torch.tensor(weights["norm_count"].to_numpy(), device="cuda:0")

In [31]:
means_target = X.groupby('target_category')['target'].median().reset_index().reset_index(names='original_index').rename(columns={'target':'mean_target'})
means_target['target_cat_name'] = means_target['original_index'].astype(str)+means_target['target_category'].astype(str)
means = X.groupby('wap_category')['wap_price_t-60'].median().reset_index().reset_index(names='original_index').rename(columns={'wap_price_t-60':'mean_wap'})
means['wap_cat_name'] = means['original_index'].astype(str)+means['wap_category'].astype(str)
means = means.merge(means_target,on='original_index')
means

C:\Users\Nick\AppData\Local\Temp\ipykernel_20988\340863963.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  means_target = X.groupby('target_category')['target'].median().reset_index().reset_index(names='original_index').rename(columns={'target':'mean_target'})
C:\Users\Nick\AppData\Local\Temp\ipykernel_20988\340863963.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  means = X.groupby('wap_category')['wap_price_t-60'].median().reset_index().reset_index(names='original_index').rename(columns={'wap_price_t-60':'mean_wap'})


,original_index,wap_category,mean_wap,wap_cat_name,target_category,mean_target,target_cat_name
0,0,"(-379.691, -9.36]",-14.38,"0(-379.691, -9.36]","(-385.291, -7.76]",-11.820197,"0(-385.291, -7.76]"
1,1,"(-9.36, -4.4]",-6.48,"1(-9.36, -4.4]","(-7.76, -3.77]",-5.440116,"1(-7.76, -3.77]"
2,2,"(-4.4, -1.22]",-2.69,"2(-4.4, -1.22]","(-3.77, -1.2]",-2.400279,"2(-3.77, -1.2]"
3,3,"(-1.22, 1.25]",0.01,"3(-1.22, 1.25]","(-1.2, 1.07]",-0.060201,"3(-1.2, 1.07]"
4,4,"(1.25, 4.45]",2.73,"4(1.25, 4.45]","(1.07, 3.63]",2.280474,"4(1.07, 3.63]"
5,5,"(4.45, 9.43]",6.56,"5(4.45, 9.43]","(3.63, 7.61]",5.300045,"5(3.63, 7.61]"
6,6,"(9.43, 392.99]",14.37,"6(9.43, 392.99]","(7.61, 446.07]",11.719465,"6(7.61, 446.07]"


In [32]:
ohe = OneHotEncoder(sparse_output=False)
ohe_out = ohe.fit_transform(
    X["wap_category"].fillna(X['wap_category'][0]).to_numpy().reshape(-1, 1),
)
X["wap_target_OHE"] = [x for x in ohe_out]

ohe = OneHotEncoder(sparse_output=False)
ohe_out = ohe.fit_transform(
    X["target_category"].fillna(X['target_category'][0]).to_numpy().reshape(-1, 1),
)
X["target_OHE"] = [x for x in ohe_out]

In [33]:
ohe = OneHotEncoder(sparse_output=False)
ohe_out = ohe.fit_transform(
    X["wap_category2"].fillna(X['wap_category2'][0]).to_numpy().reshape(-1, 1),
)
X["wap_target_OHE2"] = [x for x in ohe_out]

ohe = OneHotEncoder(sparse_output=False)
ohe_out = ohe.fit_transform(
    X["target_category2"].fillna(X['target_category2'][0]).to_numpy().reshape(-1, 1),
)
X["target_OHE2"] = [x for x in ohe_out]

ohe = OneHotEncoder(sparse_output=False)
ohe_out = ohe.fit_transform(
    X["wap_category3"].fillna(X['wap_category3'][0]).to_numpy().reshape(-1, 1),
)
X["wap_target_OHE3"] = [x for x in ohe_out]

ohe = OneHotEncoder(sparse_output=False)
ohe_out = ohe.fit_transform(
    X["target_category3"].fillna(X['target_category3'][0]).to_numpy().reshape(-1, 1),
)
X["target_OHE3"] = [x for x in ohe_out]

In [34]:
import pandas as pd
import numpy as np


# Function to blur the one-hot encoded vectors
def blur_vector(vector, blur_factor=0.2):
    idx = np.argmax(vector)  # Get the index of the one-hot class
    blurred_vector = np.zeros_like(vector)
    
    blurred_vector[idx] += (1 - blur_factor)  # Highest probability to the original class
    
    # Distribute the blur_factor to the neighbors without wrapping around
    for i in range(1, len(vector)):
        left = idx - i
        right = idx + i
        
        if left >= 0:
            blurred_vector[left] += blur_factor / i
        if right < len(vector):
            blurred_vector[right] += blur_factor / i
        if left < 0 and right >= len(vector):
            break
    
    return blurred_vector


In [35]:
X["target_OHE"] = X["target_OHE"].apply(blur_vector)
X["wap_target_OHE"] = X["wap_target_OHE"].apply(blur_vector)

In [36]:
X["target_OHE2"] = X["target_OHE2"].apply(blur_vector)
X["wap_target_OHE2"] = X["wap_target_OHE2"].apply(blur_vector)
X["target_OHE3"] = X["target_OHE3"].apply(blur_vector)
X["wap_target_OHE3"] = X["wap_target_OHE3"].apply(blur_vector)

In [34]:
importlib.reload(utils.torch_classes)
trading_data = utils.torch_classes.TradingData(X,means)
hidden_size = 64
# trading_data.generate_batches()

  0%|          | 0/95236 [00:00<?, ?it/s]c:\Users\Nick\Documents\GitHub\OptiverKaggle\utils\torch_classes.py:54: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  self.stocksDict[stock_id].wap_daily_ohe[day] = torch.tensor(stock_daily_data['wap_target_OHE'].to_list(), requires_grad=False, device='cuda:0')
 10%|█         | 9544/95236 [00:22<02:46, 515.05it/s]

Missing Targets for day=438, for stock_id=19, Excluding


 50%|████▉     | 47442/95236 [01:52<01:51, 429.53it/s]

Missing Targets for day=328, for stock_id=101, Excluding


 64%|██████▍   | 61099/95236 [02:35<01:29, 379.98it/s]

...

 76%|███████▌  | 72529/95236 [03:18<00:45, 498.62it/s] 

Missing Targets for day=388, for stock_id=158, Excluding


100%|██████████| 95236/95236 [04:09<00:00, 382.11it/s]


Finished processing stock data


100%|██████████| 481/481 [00:03<00:00, 135.90it/s]


In [35]:
trading_data.generate_batches()

Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 237.18it/s]


In [36]:
torch.cuda.empty_cache()
gc.collect()

0

In [37]:
importlib.reload(utils.torch_classes)
importlib.reload(utils.training_testing)

<module 'utils.training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\utils\\training_testing.py'>

In [38]:
optim_dict = {
    "RMSProp": optim.RMSprop,
    "Adam": optim.Adam,
    "RAdam": optim.RAdam,
    "NAdam": optim.NAdam,
    "AdamW": optim.AdamW,
    "SGD": optim.SGD,
    "Rprop": optim.Rprop,
}

In [39]:
def model_pipeline(
    trading_df=trading_data, config=None, prev_model_file=None, prev_model_version=450
):
    trading_df = trading_data
    with wandb.init(project="Optviver_new", config=config, save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        wandb.define_metric("L1_loss_wap_epoch", summary="min")
        wandb.define_metric("Accuracy", summary="max")
        config = wandb.config
        wandb.run.log_code("utils")

        input_size = len(trading_df.stocksDict[0].data_daily[0][0])

        target_size_ohe = len(trading_df.stocksDict[0].target_daily_ohe[0][0])

        print(target_size_ohe)
        
        model_ohe = utils.torch_classes.GRUNetV5_a(
            input_size,
            config["hidden_size"],
            num_layers=config["num_layers"],
            fc0_size=config["fc0_size"],
            remove_first_linear=config['remove_first_linear'],
            detach = config['detach'],
            target_size=target_size_ohe
        ).to("cuda:0")

        model_reg = utils.torch_classes.GRUNetV5_b(
            input_size,
            config["hidden_size"],
            num_layers=config["num_layers"],
            fc0_size=config["fc0_size"],
            remove_first_linear=config['remove_first_linear'],
            detach = config['detach'],
            target_size=target_size_ohe
        ).to("cuda:0")

        wandb.watch(model_ohe, log='all') 
        optimizer_ohe = optim_dict[config['optim']](model_ohe.parameters(), lr=config['learning_rate'], weight_decay=0.0001)

        wandb.watch(model_ohe, log='all') 
        # optimizer_reg= optim_dict[config['optim']](model_reg.parameters(), lr=config['learning_rate'], weight_decay=0.01)
        optimizer_reg = optim.AdamW(model_reg.parameters(), lr=config['learning_rate_reg'], weight_decay=0.0001)  
        
        config = wandb.config
        print(config)
        print(config['ohe_targets'])
        if prev_model_file != None:
            model_name = prev_model_file
            config['prev_model_file'] = prev_model_file            
            model_loc = f"models/{model_name}/{model_name}_{prev_model_version}.pt"
            model_data = torch.load(model_loc, map_location=torch.device("cuda:0"))
            print(model_data["model_state_dict"].keys())
            print(model_data.keys())

            # del_keys = ['fc_final.weight', 'fc_final.bias', 'fc_wap0.weight']
            # [model_data['model_state_dict'].pop(k) for k in del_keys]
            # model.load_state_dict(model_data["model_state_dict"], strict=False)
            # optimizer.load_state_dict(model_data["optim"])

        print(model_ohe)
        print(model_reg)
        trading_df.reset_hidden(
            hidden_size=config["hidden_size"], num_layers=config["num_layers"]
        )
        criterion = nn.CrossEntropyLoss()
        print(criterion)
        print(optimizer_reg)
        print(optimizer_ohe)    
        output = utils.training_testing_dual_optim.train_model(
            trading_df, model_ohe,model_reg, config, optimizer_ohe,optimizer_reg, criterion
        )

    return output


# arroios

In [58]:
importlib.reload(utils.torch_classes)
importlib.reload(utils.training_testing)
importlib.reload(utils.training_testing_double)
importlib.reload(utils.training_testing_dual_optim)


<module 'utils.training_testing_dual_optim' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\utils\\training_testing_dual_optim.py'>

In [63]:
config_static = {
    "learning_rate": 0.0001,
    "learning_rate_reg": 0.00001,
    "hidden_size": 128,
    "num_layers": 2,
    "batch_norm": 1,
    "epochs": 5000,
    "mini_batches": 30,
    "fc0_size": 128,
    "note": "New Split GRU5,confirming good results from silver plsama",
    'optim': 'RMSProp',
    'ohe_targets':means,
    'ohe_loss_weight':0.77,
    "l1_squared": 1,
    "detach": 1,
    "ohe_squared": 0.5,
    'weigh_dif_factor': 2,
    'remove_first_linear':False,
}
config = config_static
torch.cuda.empty_cache()
trading_data.detach_hidden()

In [42]:
len(stat_cols)

23

In [64]:
output = model_pipeline(trading_data, config_static)

7
{'learning_rate': 0.0001, 'learning_rate_reg': 1e-05, 'hidden_size': 128, 'num_layers': 2, 'batch_norm': 1, 'epochs': 5000, 'mini_batches': 30, 'fc0_size': 128, 'note': 'New Split GRU5,confirming good results from silver plsama', 'optim': 'RMSProp', 'ohe_targets': '   original_index       wap_category  mean_wap        wap_cat_name  \\\n0               0  (-379.691, -9.36]    -14.38  0(-379.691, -9.36]   \n1               1      (-9.36, -4.4]     -6.48      1(-9.36, -4.4]   \n2               2      (-4.4, -1.22]     -2.69      2(-4.4, -1.22]   \n3               3      (-1.22, 1.25]      0.01      3(-1.22, 1.25]   \n4               4       (1.25, 4.45]      2.73       4(1.25, 4.45]   \n5               5       (4.45, 9.43]      6.56       5(4.45, 9.43]   \n6               6     (9.43, 392.99]     14.37     6(9.43, 392.99]   \n\n     target_category  mean_target     target_cat_name  \n0  (-385.291, -7.76]   -11.820197  0(-385.291, -7.76]  \n1     (-7.76, -3.77]    -5.440116     1(-7.76, 

  0%|          | 0/5000 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\dxog9d5v.table.json'
[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\ppv4yjec.table.json'
[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\t79v6o8w.table.json'
[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\aseddmz3.table.json'
[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\k6oen7dj.table.json'


Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_92576\1574468590.py", line 70, in model_pipeline
    output = utils.training_testing_dual_optim.train_model(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\utils\training_testing_dual_optim.py", line 169, in train_model
    epoch_loss_ohe.backward(retain_graph=True)
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\autograd\__init__.py", line 251, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt


Accuracy_wap,▁▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████████
L1_loss_wap_epoch,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_loss_ohe,█▅▇▅▅▆▄▅▅▄▂▄▄▂▅▄▂▅▂▃▄▃▅▅▃▅▄▃▁▃▄▂▃▁▃▄▁▄▄▃
epoch_loss_reg,▅▅▅▃▆█▄▅▅▅▁▃▄▂█▅▅▅▁▆█▄█▄▅▅▃▄▁▄▄▅▅▁▆█▁█▄▅
losst_to_zero,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
output_sd,█▅▄▃▃▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂
relu_sum,█▇▇▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
target_calc_loss,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
train_class_tgt_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_class_wap_loss,█▇▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


KeyboardInterrupt: 

In [65]:
sweep_config = {"method": "bayes"}

metric = {"name": "L1_loss_wap_epoch.min", "goal": "minimize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optim": {"values": ["RMSProp", "AdamW"]},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "num_layers": {"values": [2]},
    "hidden_size": {"values": [32,64,128,256]},
    "fc0_size": {"values": [128,256]},
    "epochs": {"value": 200},
    "mini_batches": {"value": 10},
    'ohe_targets': {"value": 'ahhh'},
    'ohe_loss_weight': {"max": 1.0, "min": 0.0001},
    'learning_rate': {"values": [0.0001,0.00005,0.00001,0.000005]},
    'learning_rate_reg': {"values": [0.0001,0.00005,0.00001,0.000005]},
    "l1_squared": {"values": [0.5,1,2]},
    "detach": {"values": [0,1,2]},
    "ohe_squared": {"values": [0.5,1,2]},
    'weigh_dif_factor': {"values": [0,1,2]},
    'remove_first_linear':{"values":[True,False]},
    "warmup": {"values": [0,5,10,20,30,100]},
}

sweep_config["parameters"] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="New_final_sweeps_Optiver")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: ae878ias
Sweep URL: https://wandb.ai/nickojelly/New_final_sweeps_Optiver/sweeps/ae878ias


wandb: Agent Starting Run: e872hguf with config:
wandb: 	detach: 2
wandb: 	epochs: 200
wandb: 	f0_layer_size: 128
wandb: 	f1_layer_size: 64
wandb: 	fc0_size: 256
wandb: 	hidden_size: 32
wandb: 	l1_squared: 0.5
wandb: 	learning_rate: 5e-05
wandb: 	learning_rate_reg: 0.0001
wandb: 	mini_batches: 10
wandb: 	num_layers: 2
wandb: 	ohe_loss_weight: 0.8508377954827773
wandb: 	ohe_squared: 2
wandb: 	ohe_targets: ahhh
wandb: 	optim: RMSProp
wandb: 	remove_first_linear: True
wandb: 	warmup: 10
wandb: 	weigh_dif_factor: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


7
{'detach': 2, 'epochs': 200, 'f0_layer_size': 128, 'f1_layer_size': 64, 'fc0_size': 256, 'hidden_size': 32, 'l1_squared': 0.5, 'learning_rate': 5e-05, 'learning_rate_reg': 0.0001, 'mini_batches': 10, 'num_layers': 2, 'ohe_loss_weight': 0.8508377954827773, 'ohe_squared': 2, 'ohe_targets': 'ahhh', 'optim': 'RMSProp', 'remove_first_linear': True, 'warmup': 10, 'weigh_dif_factor': 0}
ahhh
GRUNetV5_a(
  (gru): GRU(23, 32, num_layers=2, batch_first=True, dropout=0.3)
  (relu): ReLU()
  (batch_norm): BatchNorm1d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (layer_norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (layer_norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (drop): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=32, out_features=256, bias=True)
  (fc_final): Linear(in_features=256, out_features=7, bias=True)
  (fc_reg0): Linear(in_features=14, out_fe

  0%|          | 0/200 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\7q38sj5k.table.json'
[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\vrarhtrz.table.json'
[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\ntc6em4w.table.json'
Epoch 00027: reducing learning rate of group 0 to 5.0000e-05.
[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\h347d8hx.table.json'
Epoch 00038: reducing learning rate of group 0 to 2.5000e-05.
5e-05
0.0001
Finished Early on reg


...

Accuracy_wap,▁▁▂▂▃▃▄▄▅▅▅▆▆▆▇▇▇▇▇▇█████████████
L1_loss_wap_epoch,▆▆▆▅▅▅▄▃▂▁▁▁▁▁▁▁▂▂▃▃▄▄▅▅▆▆▇▇▇▇███
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
epoch_loss_ohe,▇█▅▇▆█▇▄▇▆▅▅▅▃▅▅▅▄▂▆▄▄▃▂▂▃▄▃▃▂▁▄▃▃▃▁▃▄▃▃
epoch_loss_reg,▅▆▁▃▂██▁▄▃▂▃▆▁▄▂▅▅▃▄▄▃▂▃▂▂▃▃▅▂▁▃▂▅▆▁▂▂▄▄
losst_to_zero,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
output_sd,█▇▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
relu_sum,███▇▇▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
target_calc_loss,▁▁▁▁▁▁▁▁▂▂▂▃▃▄▄▅▅▆▆▆▇▇▇▇▇████████
train_class_tgt_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_class_wap_loss,▆██▇▇▆▆▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: 0du0hwpi with config:
wandb: 	detach: 0
wandb: 	epochs: 200
wandb: 	f0_layer_size: 128
wandb: 	f1_layer_size: 64
wandb: 	fc0_size: 256
wandb: 	hidden_size: 256
wandb: 	l1_squared: 2
wandb: 	learning_rate: 1e-05
wandb: 	learning_rate_reg: 1e-05
wandb: 	mini_batches: 10
wandb: 	num_layers: 2
wandb: 	ohe_loss_weight: 0.35410106635315547
wandb: 	ohe_squared: 2
wandb: 	ohe_targets: ahhh
wandb: 	optim: RMSProp
wandb: 	remove_first_linear: False
wandb: 	warmup: 20
wandb: 	weigh_dif_factor: 2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


7
{'detach': 0, 'epochs': 200, 'f0_layer_size': 128, 'f1_layer_size': 64, 'fc0_size': 256, 'hidden_size': 256, 'l1_squared': 2, 'learning_rate': 1e-05, 'learning_rate_reg': 1e-05, 'mini_batches': 10, 'num_layers': 2, 'ohe_loss_weight': 0.35410106635315547, 'ohe_squared': 2, 'ohe_targets': 'ahhh', 'optim': 'RMSProp', 'remove_first_linear': False, 'warmup': 20, 'weigh_dif_factor': 2}
ahhh
GRUNetV5_a(
  (gru): GRU(256, 256, num_layers=2, batch_first=True, dropout=0.3)
  (relu): ReLU()
  (batch_norm): BatchNorm1d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (layer_norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (layer_norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (drop): Dropout(p=0.5, inplace=False)
  (fc0): Linear(in_features=23, out_features=256, bias=True)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc_final): Linear(in_features=256, out_

  0%|          | 0/200 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmpduqf0sibwandb-media\\82zj1ow6.table.json'
